# **Ejercicios de pair programming Módulo 2 Sprint 2**
## **ETL: Lección 3**
### **Transformación: Clases y funciones de limpieza**

---

En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" 

import requests
import pandas as pd
import numpy as np

In [2]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [28]:
df_censo_ccaa = pd.read_csv("data/poblacion_comunidades.csv", index_col = 0)

In [25]:
class Extraccion:
    
    def __init__(self, año_inicio, año_final):
        
        self.año_inicio = año_inicio
        self.año_final = año_final
        
    def nacional(self):  
        df_energia_nacional = pd.DataFrame()
        for año in range(self.año_inicio, self.año_final + 1):

            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year"
            response = requests.get(url=url)

            fichero_json_renovable = response.json()["included"][0]["attributes"]["values"] 
            fichero_json_no_renovable = response.json()["included"][1]["attributes"]["values"]

            df_renovables = pd.json_normalize(fichero_json_renovable)
            df_no_renovables = pd.json_normalize(fichero_json_no_renovable)

            df_renovables["tipo"] = response.json()["included"][0]["type"]
            df_no_renovables["tipo"] = response.json()["included"][1]["type"]
            
            df_energia_nacional = pd.concat([df_energia_nacional, df_renovables, df_no_renovables], axis = 0)
            
        return df_energia_nacional
    
    
    def ccaa(self, cod_comunidades):  
        
        self.cod_comunidades = cod_comunidades
        
        df_completo_ccaa = pd.DataFrame()
        for año in range(self.año_inicio, self.año_final + 1):

            for k, v in cod_comunidades.items():
                
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={año}-01-01T00:00&end_date={año}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
            
                response2 = requests.get(url=url)

                if len(response2.json()["included"]) == 1:

                    fichero_json_no_renovable = response2.json()["included"][0]["attributes"]["values"] 
                    df_no_renovables = pd.json_normalize(fichero_json_no_renovable)
                    df_no_renovables["tipo"] = response2.json()["included"][0]["type"]
                    df_no_renovables["comunidad"] = k
                    df_no_renovables["id_comunidad"] = v  
                              
                    df_completo_ccaa = pd.concat([df_completo_ccaa, df_no_renovables], axis = 0)

                elif len(response2.json()["included"]) == 2:

                    fichero_json_no_renovable = response2.json()["included"][0]["attributes"]["values"] 
                    df_no_renovables = pd.json_normalize(fichero_json_no_renovable)
                    df_no_renovables["tipo"] = response2.json()["included"][0]["type"]
                    df_no_renovables["comunidad"] = k
                    df_no_renovables["id_comunidad"] = v           

                    fichero_json_renovable = response2.json()["included"][1]["attributes"]["values"] 
                    df_renovables = pd.json_normalize(fichero_json_renovable)
                    df_renovables["tipo"] = response2.json()["included"][1]["type"]
                    df_renovables["comunidad"] = k
                    df_renovables["id_comunidad"] = v   

                    df_completo_ccaa = pd.concat([df_completo_ccaa, df_no_renovables, df_renovables], axis = 0)


     
        return df_completo_ccaa
        
    def limpieza_censo(self, df):
        
        
        df.drop(["Comunidades_y_Ciudades_Autónomas"], axis =1, inplace = True)
        
        return df
        
    
    def limpieza_energias(self, df_energias):
        
        df_energias["value"]=df_energias["value"].round(2)
        df_energias["percentage"]=df_energias["percentage"].round(2)
        df_energias[["fecha","hora"]]=df_energias.datetime.str.split("T",expand = True)
        df_energias.drop(["hora"], axis = 1, inplace=True)
        df_energias["fecha"]=df_energias["fecha"].apply(pd.to_datetime)
        
        return df_energias
       
        
        

In [26]:
energias = Extraccion(2011, 2022)

In [22]:
df_nacional = energias.nacional()

In [23]:
df_ccaa = energias.ccaa(cod_comunidades) 

In [29]:
energias.limpieza_censo(df_censo_ccaa)

,Comunidad,indice,Año,Total
0,Ceuta,8744,2022,83.117
1,Ceuta,8744,2021,83.517
2,Ceuta,8744,2020,84.202
3,Ceuta,8744,2019,84.777
4,Ceuta,8744,2018,85.144
...,...,...,...,...
470,La Rioja,20,2002,281.614
471,La Rioja,20,2001,270.400
472,La Rioja,20,2000,264.178
473,La Rioja,20,1999,265.178


In [30]:
energias.limpieza_energias(df_nacional)

,value,percentage,datetime,tipo,fecha
0,8.666705e+07,0.31,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01
0,1.926868e+08,0.69,2011-01-01T00:00:00.000+01:00,No renovable,2011-01-01
0,8.533872e+07,0.30,2012-01-01T00:00:00.000+01:00,Renovable,2012-01-01
0,1.977799e+08,0.70,2012-01-01T00:00:00.000+01:00,No renovable,2012-01-01
0,1.097577e+08,0.40,2013-01-01T00:00:00.000+01:00,Renovable,2013-01-01
0,1.635286e+08,0.60,2013-01-01T00:00:00.000+01:00,No renovable,2013-01-01
0,1.078761e+08,0.40,2014-01-01T00:00:00.000+01:00,Renovable,2014-01-01
0,1.586356e+08,0.60,2014-01-01T00:00:00.000+01:00,No renovable,2014-01-01
0,9.408811e+07,0.35,2015-01-01T00:00:00.000+01:00,Renovable,2015-01-01
0,1.733657e+08,0.65,2015-01-01T00:00:00.000+01:00,No renovable,2015-01-01


In [31]:
energias.limpieza_energias(df_ccaa)

,value,percentage,datetime,tipo,comunidad,id_comunidad,fecha
0,202972.89,1.00,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-01
0,3589.08,0.02,2011-01-01T00:00:00.000+01:00,Renovable,Melilla,8745,2011-01-01
0,211290.04,0.98,2011-01-01T00:00:00.000+01:00,No renovable,Melilla,8745,2011-01-01
0,10996921.24,0.28,2011-01-01T00:00:00.000+01:00,Renovable,Andalucía,4,2011-01-01
0,27608105.11,0.72,2011-01-01T00:00:00.000+01:00,No renovable,Andalucía,4,2011-01-01
...,...,...,...,...,...,...,...
0,6813411.80,0.80,2022-01-01T00:00:00.000+01:00,No renovable,Canarias,8742,2022-01-01
0,2670445.81,0.23,2022-01-01T00:00:00.000+01:00,Renovable,Región de Murcia,21,2022-01-01
0,8982495.46,0.77,2022-01-01T00:00:00.000+01:00,No renovable,Región de Murcia,21,2022-01-01
0,1106679.16,0.48,2022-01-01T00:00:00.000+01:00,Renovable,La Rioja,20,2022-01-01


In [33]:
df_nacional.to_csv("data/energia_nacional.csv")
df_ccaa.to_csv("data/energia_ccaa.csv")
df_censo_ccaa.to_csv("data/censo_ccaa.csv")